# Wprowadzenie do inżynierii promptów
Inżynieria promptów to proces projektowania i optymalizacji promptów do zadań związanych z przetwarzaniem języka naturalnego. Obejmuje wybór odpowiednich promptów, dostosowywanie ich parametrów oraz ocenę ich skuteczności. Inżynieria promptów jest kluczowa dla osiągnięcia wysokiej dokładności i wydajności modeli NLP. W tej sekcji przyjrzymy się podstawom inżynierii promptów, korzystając z modeli OpenAI do eksploracji.


### Ćwiczenie 1: Tokenizacja
Poznaj tokenizację z użyciem tiktoken, szybkiego tokenizatora open-source od OpenAI
Zobacz [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) po więcej przykładów.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Ćwiczenie 2: Sprawdź konfigurację klucza Github Models

Uruchom poniższy kod, aby upewnić się, że Twój endpoint Github Models został poprawnie skonfigurowany. Kod po prostu wysyła prostą podstawową podpowiedź i sprawdza odpowiedź. Wprowadzenie `oh say can you see` powinno zostać dokończone w stylu `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Ćwiczenie 3: Zmyślenia
Sprawdź, co się dzieje, gdy poprosisz LLM o wygenerowanie odpowiedzi na temat, który może nie istnieć, albo na temat, o którym model może nie mieć wiedzy, bo był poza jego zbiorem danych treningowych (nowsze informacje). Zobacz, jak zmienia się odpowiedź, jeśli spróbujesz innego polecenia lub innego modelu.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Ćwiczenie 4: Oparte na instrukcji
Użyj zmiennej "text", aby ustawić główną treść,
a zmiennej "prompt", aby podać instrukcję związaną z tą treścią.

Tutaj prosimy model, aby podsumował tekst w sposób zrozumiały dla ucznia drugiej klasy szkoły podstawowej.


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Ćwiczenie 5: Złożona podpowiedź  
Spróbuj zapytania, które zawiera wiadomości systemowe, użytkownika i asystenta  
System ustawia kontekst asystenta  
Wiadomości użytkownika i asystenta tworzą kontekst rozmowy wieloturnej

Zwróć uwagę, że osobowość asystenta została ustawiona na „sarkastyczną” w kontekście systemowym.  
Spróbuj użyć innego kontekstu osobowości. Albo wypróbuj inną serię wiadomości wejściowych/wyjściowych


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Ćwiczenie: Zbadaj swoją intuicję
Powyższe przykłady pokazują schematy, które możesz wykorzystać do tworzenia własnych promptów (prostych, złożonych, instrukcji itp.) – spróbuj wymyślić inne ćwiczenia, aby sprawdzić pozostałe pomysły, o których rozmawialiśmy, takie jak przykłady, wskazówki i inne.



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony przy użyciu usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby zapewnić poprawność tłumaczenia, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło nadrzędne. W przypadku informacji krytycznych zalecane jest skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
